<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#평가방식-MAE" data-toc-modified-id="평가방식-MAE-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>평가방식 MAE</a></span></li><li><span><a href="#library-import" data-toc-modified-id="library-import-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>library import</a></span></li><li><span><a href="#data-load" data-toc-modified-id="data-load-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>data load</a></span></li><li><span><a href="#전처리" data-toc-modified-id="전처리-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>전처리</a></span></li><li><span><a href="#optuna" data-toc-modified-id="optuna-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>optuna</a></span></li><li><span><a href="#model-fitting-with-tuned-hyperparameters" data-toc-modified-id="model-fitting-with-tuned-hyperparameters-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>model fitting with tuned hyperparameters</a></span></li></ul></div>

# 평가방식 MAE
... 회귀.... <br>
회귀도 input은 수치여야함.

# library import

In [1]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# data load

In [2]:
train = pd.read_csv('../235743_구내식당 식사 인원 예측 AI 경진대회_data/train.csv')
test = pd.read_csv('../235743_구내식당 식사 인원 예측 AI 경진대회_data/test.csv')

In [3]:
train.head(6)
# 일자
# 요일 ###
# 본사 정원수
# 본사휴가자수
# 본사출장자수
# 본사시간외근무명령서승인건수
# 현본사소속재택근무자수 ###
# 조식메뉴
# 중식메뉴 ###
# 석식메뉴 ###

# 중식계
# 석식계

# 중식계, 석식계는 합계인듯. 예측해야 하는 것.

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0
5,2016-02-11,목,2601,383,143,417,0.0,팬케익/찐빵 우유/두유/주스 계란후라이 견과류죽/쌀밥 (쌀:국내산) 감자찌개 ...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 시래기국 훈제오리구이 도토리묵무침 쌈무/양...","참치회덮밥 (쌀,현미흑미:국내산) 맑은국 군만두 과일샐러드 락교 포기김치 (...",1045.0,550.0


# 전처리

In [4]:
# train['조식메뉴'].value_counts()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   일자              1205 non-null   object 
 1   요일              1205 non-null   object 
 2   본사정원수           1205 non-null   int64  
 3   본사휴가자수          1205 non-null   int64  
 4   본사출장자수          1205 non-null   int64  
 5   본사시간외근무명령서승인건수  1205 non-null   int64  
 6   현본사소속재택근무자수     1205 non-null   float64
 7   조식메뉴            1205 non-null   object 
 8   중식메뉴            1205 non-null   object 
 9   석식메뉴            1205 non-null   object 
 10  중식계             1205 non-null   float64
 11  석식계             1205 non-null   float64
dtypes: float64(3), int64(4), object(5)
memory usage: 113.1+ KB


In [5]:
train.loc[(train['조식메뉴'].str.contains('쇠'))|(train['조식메뉴'].str.contains('소'))|(train['조식메뉴'].str.contains('고기'))|
          (train['조식메뉴'].str.contains('돼지'))|(train['조식메뉴'].str.contains('돈육'))|
          (train['조식메뉴'].str.contains('닭'))|(train['조식메뉴'].str.contains('치킨'))|
          (train['조식메뉴'].str.contains('오리')), '조식메뉴'] = 1
train.loc[train['조식메뉴']!=1, '조식메뉴'] = 0
test.loc[(test['조식메뉴'].str.contains('쇠'))|(test['조식메뉴'].str.contains('소'))|(test['조식메뉴'].str.contains('고기'))|
          (test['조식메뉴'].str.contains('돼지'))|(test['조식메뉴'].str.contains('돈육'))|
          (test['조식메뉴'].str.contains('닭'))|(test['조식메뉴'].str.contains('치킨'))|
          (test['조식메뉴'].str.contains('오리')), '조식메뉴'] = 1
test.loc[test['조식메뉴']!=1, '조식메뉴'] = 0

# int 형으로 바꾸기
train['조식메뉴'] = train['조식메뉴'].map(int)
test['조식메뉴'] = test['조식메뉴'].map(int)

In [6]:
# train[train['중식메뉴'].contains('고기')]
# (train.loc[train['중식메뉴'].str.contains('고기'), '중식메뉴']).count()
train.shape
# train.loc[train['중식메뉴'].str.contains('고기'), '중식메뉴']
train.loc[(train['중식메뉴'].str.contains('쇠'))|(train['중식메뉴'].str.contains('소'))|(train['중식메뉴'].str.contains('고기'))|
          (train['중식메뉴'].str.contains('돼지'))|(train['중식메뉴'].str.contains('돈육'))|
          (train['중식메뉴'].str.contains('닭'))|(train['중식메뉴'].str.contains('치킨'))|
          (train['중식메뉴'].str.contains('오리')), '중식메뉴'] = 1
train.loc[train['중식메뉴']!=1, '중식메뉴'] = 0
# train.loc[(train['중식메뉴'].str.contains('돼지'))|(train['중식메뉴'].str.contains('돈육')), '중식메뉴']
# train.loc[(train['중식메뉴'].str.contains('닭'))|(train['중식메뉴'].str.contains('치킨')), '중식메뉴']
# train.loc[(train['중식메뉴'].str.contains('오리')), '중식메뉴']

test.loc[(test['중식메뉴'].str.contains('쇠'))|(test['중식메뉴'].str.contains('소'))|(test['중식메뉴'].str.contains('고기'))|
          (test['중식메뉴'].str.contains('돼지'))|(test['중식메뉴'].str.contains('돈육'))|
          (test['중식메뉴'].str.contains('닭'))|(test['중식메뉴'].str.contains('치킨'))|
          (test['중식메뉴'].str.contains('오리')), '중식메뉴'] = 1
test.loc[test['중식메뉴']!=1, '중식메뉴'] = 0

# int 형으로 바꾸기
train['중식메뉴'] = train['중식메뉴'].map(int)
test['중식메뉴'] = test['중식메뉴'].map(int)

(1205, 12)

In [7]:
train.loc[(train['석식메뉴'].str.contains('쇠'))|(train['석식메뉴'].str.contains('소'))|(train['석식메뉴'].str.contains('고기'))|
          (train['석식메뉴'].str.contains('돼지'))|(train['석식메뉴'].str.contains('돈육'))|
          (train['석식메뉴'].str.contains('닭'))|(train['석식메뉴'].str.contains('치킨'))|
          (train['석식메뉴'].str.contains('오리')), '석식메뉴'] = 1
train.loc[train['석식메뉴']!=1, '석식메뉴'] = 0

test.loc[(test['석식메뉴'].str.contains('쇠'))|(test['석식메뉴'].str.contains('소'))|(test['석식메뉴'].str.contains('고기'))|
          (test['석식메뉴'].str.contains('돼지'))|(test['석식메뉴'].str.contains('돈육'))|
          (test['석식메뉴'].str.contains('닭'))|(test['석식메뉴'].str.contains('치킨'))|
          (test['석식메뉴'].str.contains('오리')), '석식메뉴'] = 1
test.loc[test['석식메뉴']!=1, '석식메뉴'] = 0

# int 형으로 바꾸기
train['석식메뉴'] = train['석식메뉴'].map(int)
test['석식메뉴'] = test['석식메뉴'].map(int)

In [8]:
train['요일'].values

train['요일'] = train['요일'].map({'월':1, '화':2, '수':3, '목':4, '금':5})
test['요일'] = test['요일'].map({'월':1, '화':2, '수':3, '목':4, '금':5})

train['요일']

array(['월', '화', '수', ..., '금', '월', '화'], dtype=object)

0       1
1       2
2       3
3       4
4       5
       ..
1200    3
1201    4
1202    5
1203    1
1204    2
Name: 요일, Length: 1205, dtype: int64

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   일자              1205 non-null   object 
 1   요일              1205 non-null   int64  
 2   본사정원수           1205 non-null   int64  
 3   본사휴가자수          1205 non-null   int64  
 4   본사출장자수          1205 non-null   int64  
 5   본사시간외근무명령서승인건수  1205 non-null   int64  
 6   현본사소속재택근무자수     1205 non-null   float64
 7   조식메뉴            1205 non-null   int64  
 8   중식메뉴            1205 non-null   int64  
 9   석식메뉴            1205 non-null   int64  
 10  중식계             1205 non-null   float64
 11  석식계             1205 non-null   float64
dtypes: float64(3), int64(8), object(1)
memory usage: 113.1+ KB


In [10]:
import lightgbm as lgbm
import random

# optuna
[공식 docs](https://optuna.readthedocs.io/_/downloads/en/latest/pdf/) <br>
[맨 처음 참고](https://ssoonidev.tistory.com/107) <br>
[dacon, 다수 알고리즘에 대한 optuna 코드 보유](https://dacon.io/codeshare/2876) <br>
[저장 안하려다 다른 포스팅 참고하라고 저장](https://velog.io/@dohy426/LGBM-Regressor-Optuna) <br>

In [11]:
lgbm.LGBMRegressor().get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': 'warn',
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [25]:
# ! pip install optuna

In [12]:
import optuna
# from optuna.integration.lightgbm import LightGBMTuner as LGBMTuner
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from sklearn.metrics import mean_absolute_error as MAE

def optuna_LGBM(trial: Trial, X_train_, y_train_, X_test_):
    parameters = {'objective': 'regression',
                  "random_state": 42,
                  'verbose': -1,
                  'metric': 'mae',
                  "silent": -1,
                  'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1),
                  'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
                 'max_depth': trial.suggest_int('max_depth', 8, 16),
                 'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0)}

#     parameters = {
#         'objective': 'regression',
#         'verbose': -1,
#         'metric': 'mse', 
#         'num_leaves': trial.suggest_int('num_leaves', 2, 1024, step=1, log=True), 
#         'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0),
#         'reg_alpha': trial.suggest_uniform('reg_alpha', 0.0, 1.0),
#         'reg_lambda': trial.suggest_uniform('reg_lambda', 0.0, 10.0),
#         'max_depth': trial.suggest_int('max_depth',3, 15),
#         'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
#         'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
#         'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
#         'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
#     }
    user_seed = 42
    random.seed(user_seed)
#     LGBM = lgbm.LGBMRegressor() # 이런식으로 하면 자꾸 에러남.

    optuna_model = lgbm.LGBMRegressor(**parameters)
    lgb_model = optuna_model.fit(X_train_, y_train_)
    score = MAE(y_train_, lgb_model.predict(X_train_))
    
    return score


X_train_중식 = train[['요일', '현본사소속재택근무자수', '중식메뉴']]
X_train_석식 = train[['요일', '현본사소속재택근무자수', '석식메뉴']]

y_train_중식 = train[['중식계']]
y_train_석식 = train[['석식계']]

X_test_중식 = test[['요일', '현본사소속재택근무자수', '중식메뉴']]
X_test_석식 = test[['요일', '현본사소속재택근무자수', '석식메뉴']]


study_중식 = optuna.create_study(direction = 'minimize', sampler = TPESampler(seed = 42))
study_석식 = optuna.create_study(direction = 'minimize', sampler = TPESampler(seed = 42))


study_중식.optimize(lambda trial : optuna_LGBM(trial, X_train_중식, y_train_중식, X_test_중식), n_trials = 50)
best_parameters_중식 = study_중식.best_trial.params
print(f'중식 Best trial: score {study_중식.best_trial.value}, \nparams {best_parameters_중식}')

study_석식.optimize(lambda trial : optuna_LGBM(trial, X_train_석식, y_train_석식, X_test_석식), n_trials = 50)
best_parameters_석식 = study_석식.best_trial.params
print(f'석식 Best trial: score {study_석식.best_trial.value}, \nparams {best_parameters_석식}')

# [scoring method name](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)

[I 2022-07-06 01:47:15,401] A new study created in memory with name: no-name-9cd996a6-7ab6-40bb-85d1-c23faf4bf2ce
[I 2022-07-06 01:47:15,403] A new study created in memory with name: no-name-9362a8a7-8613-4f83-8bfa-960f8271a1bb
[I 2022-07-06 01:47:24,132] Trial 0 finished with value: 86.60018953843539 and parameters: {'learning_rate': 0.037460266483547784, 'n_estimators': 4759, 'max_depth': 14, 'reg_alpha': 0.24810409748678125}. Best is trial 0 with value: 86.60018953843539.
[I 2022-07-06 01:47:25,361] Trial 1 finished with value: 90.38550981573029 and parameters: {'learning_rate': 0.01561030385783923, 'n_estimators': 864, 'max_depth': 8, 'reg_alpha': 2.9154431891537547}. Best is trial 0 with value: 86.60018953843539.
[I 2022-07-06 01:47:30,211] Trial 2 finished with value: 86.53539358509452 and parameters: {'learning_rate': 0.06011549002420346, 'n_estimators': 3570, 'max_depth': 8, 'reg_alpha': 7.579479953348009}. Best is trial 2 with value: 86.53539358509452.
[I 2022-07-06 01:47:32,2

[I 2022-07-06 01:50:36,160] Trial 31 finished with value: 84.82270598322599 and parameters: {'learning_rate': 0.08943579408927509, 'n_estimators': 4561, 'max_depth': 16, 'reg_alpha': 0.002607289571006883}. Best is trial 29 with value: 84.69772142786506.
[I 2022-07-06 01:50:45,430] Trial 32 finished with value: 85.0409613024973 and parameters: {'learning_rate': 0.0805561138075131, 'n_estimators': 4586, 'max_depth': 16, 'reg_alpha': 0.018602136449551845}. Best is trial 29 with value: 84.69772142786506.
[I 2022-07-06 01:50:54,242] Trial 33 finished with value: 85.11149523941599 and parameters: {'learning_rate': 0.09110599355259574, 'n_estimators': 3963, 'max_depth': 16, 'reg_alpha': 0.0030171788486963846}. Best is trial 29 with value: 84.69772142786506.
[I 2022-07-06 01:51:04,647] Trial 34 finished with value: 84.63796861101876 and parameters: {'learning_rate': 0.09139473647545716, 'n_estimators': 4961, 'max_depth': 15, 'reg_alpha': 0.00865572499516544}. Best is trial 34 with value: 84.63

중식 Best trial: score 84.53402057120758, 
params {'learning_rate': 0.09399761729133053, 'n_estimators': 4943, 'max_depth': 15, 'reg_alpha': 0.034521570434257846}


[I 2022-07-06 01:53:24,525] Trial 0 finished with value: 71.87424406166356 and parameters: {'learning_rate': 0.037460266483547784, 'n_estimators': 4759, 'max_depth': 14, 'reg_alpha': 0.24810409748678125}. Best is trial 0 with value: 71.87424406166356.
[I 2022-07-06 01:53:25,652] Trial 1 finished with value: 75.24753307699703 and parameters: {'learning_rate': 0.01561030385783923, 'n_estimators': 864, 'max_depth': 8, 'reg_alpha': 2.9154431891537547}. Best is trial 0 with value: 71.87424406166356.
[I 2022-07-06 01:53:29,983] Trial 2 finished with value: 71.65020090140202 and parameters: {'learning_rate': 0.06011549002420346, 'n_estimators': 3570, 'max_depth': 8, 'reg_alpha': 7.579479953348009}. Best is trial 2 with value: 71.65020090140202.
[I 2022-07-06 01:53:31,803] Trial 3 finished with value: 73.05916430268788 and parameters: {'learning_rate': 0.08324593965363418, 'n_estimators': 1140, 'max_depth': 9, 'reg_alpha': 0.00541524411940254}. Best is trial 2 with value: 71.65020090140202.
[I

[I 2022-07-06 01:55:59,946] Trial 31 finished with value: 70.22571172728206 and parameters: {'learning_rate': 0.0908887950426344, 'n_estimators': 4545, 'max_depth': 14, 'reg_alpha': 0.1693009408358799}. Best is trial 27 with value: 70.14369944599827.
[I 2022-07-06 01:56:07,371] Trial 32 finished with value: 70.2096710683453 and parameters: {'learning_rate': 0.08929286501465403, 'n_estimators': 4627, 'max_depth': 14, 'reg_alpha': 0.16442811821490919}. Best is trial 27 with value: 70.14369944599827.
[I 2022-07-06 01:56:14,958] Trial 33 finished with value: 70.23997491748493 and parameters: {'learning_rate': 0.08547217258711996, 'n_estimators': 4684, 'max_depth': 15, 'reg_alpha': 0.09844831596765403}. Best is trial 27 with value: 70.14369944599827.
[I 2022-07-06 01:56:23,192] Trial 34 finished with value: 70.53487417268954 and parameters: {'learning_rate': 0.06829378581935742, 'n_estimators': 4964, 'max_depth': 14, 'reg_alpha': 0.02737334387978021}. Best is trial 27 with value: 70.1436994

석식 Best trial: score 69.94227081390027, 
params {'learning_rate': 0.09409185256453527, 'n_estimators': 4966, 'max_depth': 13, 'reg_alpha': 0.007793663772671272}


# model fitting with tuned hyperparameters
중식 Best trial: score 84.53402057120758, <br>
params {'learning_rate': 0.09399761729133053, 'n_estimators': 4943, 'max_depth': 15, 'reg_alpha': 0.034521570434257846} <br>

석식 Best trial: score 69.94227081390027, <br>
params {'learning_rate': 0.09409185256453527, 'n_estimators': 4966, 'max_depth': 13, 'reg_alpha': 0.007793663772671272} <br>

In [13]:
X_train_중식 = train[['요일', '현본사소속재택근무자수', '중식메뉴']]
X_train_석식 = train[['요일', '현본사소속재택근무자수', '석식메뉴']]
y_train_중식 = train[['중식계']]
y_train_석식 = train[['석식계']]

X_test_중식 = test[['요일', '현본사소속재택근무자수', '중식메뉴']]
X_test_석식 = test[['요일', '현본사소속재택근무자수', '석식메뉴']]

user_seed = 42
random.seed(user_seed)
LGBM_중식 = lgbm.LGBMRegressor(random_state = user_seed, learning_rate= 0.09399761729133053, n_estimators= 4943, max_depth= 15, reg_alpha= 0.034521570434257846)
LGBM_석식 = lgbm.LGBMRegressor(random_state = user_seed, learning_rate= 0.09409185256453527, n_estimators= 4966, max_depth= 13, reg_alpha= 0.007793663772671272)

model_중식 = LGBM_중식.fit(X_train_중식, y_train_중식) # fit에는 하이퍼파리미터 따로 적용 못하나봄. 계속 여기다 hp 집어넣을 때마다 에러남.
y_pred_test_중식 = model_중식.predict(X_test_중식, num_iteration = LGBM_중식.best_iteration_)

model_석식 = LGBM_석식.fit(X_train_석식, y_train_석식)
y_pred_test_석식 = model_석식.predict(X_test_석식, num_iteration = LGBM_석식.best_iteration_)


In [14]:
중식계 = pd.DataFrame(y_pred_test_중식)
중식계.head()

,0
0,1141.521689
1,927.314850
2,735.726085
3,1143.970130
4,882.964365


In [15]:
석식계 = pd.DataFrame(y_pred_test_석식)
석식계.head()

,0
0,509.911800
1,455.554914
2,263.620759
3,525.557363
4,538.073286


In [16]:
findata = pd.concat([test['일자'], 중식계, 석식계], axis = 1)
findata.head()

,일자,0,0
0,2021-01-27,1141.521689,509.911800
1,2021-01-28,927.314850,455.554914
2,2021-01-29,735.726085,263.620759
3,2021-02-01,1143.970130,525.557363
4,2021-02-02,882.964365,538.073286


In [17]:
findata.columns = ['일자', '중식계', '석식계']
findata.head()

,일자,중식계,석식계
0,2021-01-27,1141.521689,509.911800
1,2021-01-28,927.314850,455.554914
2,2021-01-29,735.726085,263.620759
3,2021-02-01,1143.970130,525.557363
4,2021-02-02,882.964365,538.073286


In [18]:
findata.to_csv('8th optuna hp 조정 수 줄임 요일, 재택자수, 중식메뉴, 석식메뉴.csv', index=False)